
<img src="img/viu_logo.png" width="200">

## 01MIAR - Análisis de Imágenes

![logo](img/python_logo.png)

*Ivan Fuertes*

# Sumario
- OpenCV
- Cargar/Mostrar imagen
- Operaciones elementales
- Blending
- Border detection
- Face detection

## OpenCV
- Librería para visión por ordenador
- Análisis de imágenes
- https://opencv.org/releases/

In [ ]:
import cv2
print(cv2.__version__)

## Cargar / Mostrar

In [ ]:
from os import path
people_path = path.join("res", "people.jpg")

In [ ]:
# Cargar imagen
img = cv2.imread(people_path)

In [ ]:
# Leer Pixel
img[0,0]

In [ ]:
# Mostrar ventana
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyWindow('Image')

- Color RGB, Red Green Blue
- Componentes entre 0 y 255
- OpenCV almacena canales en BGR por defecto

In [ ]:
# Descomponer imagen en componentes
b, g, r = cv2.split(img)
print(type(b))
print(b)

In [ ]:
# Recomponer
img_alt = cv2.merge((b,r,g))

- Guardar imagen

In [ ]:
people_alt_path = path.join("res", "o_people_alt.jpg")
cv2.imwrite(people_alt_path, img_alt)

- Operaciones con dos imágenes

In [ ]:
color_path = path.join("res", "overlay.jpg")
img_color = cv2.imread(color_path)

In [ ]:
# Sumar dos imágenes
# img_add = cv2.add(img, img_color)
img_add = img + img_color

cv2.imshow('Image', img_add)
cv2.waitKey(0)
cv2.destroyWindow('Image')

In [ ]:
# Restar dos imágenes
# img_sub = cv2.subtract(img, img_color)
img_sub = img - img_color

cv2.imshow('Image', img_sub)
cv2.waitKey(0)
cv2.destroyWindow('Image')

## Blending
- img = alpha * img1 + (1 - alpha) * img2
- 0 > alpha > 1
- addweighted
  - dst = img1 * alpha + img2 * beta + gamma

In [ ]:
# Blending de dos imágenes
img_blend = cv2.addWeighted(img, 0.3, img_color, 0.7, 0)

cv2.imshow('Image', img_blend)
cv2.waitKey(0)
cv2.destroyWindow('Image')

## Detección de bordes
- Filtros de imágenes, convoluciones
- Teoría del gradiente de imagen

<img src="img/image_gradient.png" width="600">

- Filtros Sobel y Laplacian

In [ ]:
# Mostrar imágenes
import matplotlib.pyplot as plt

def show_image(sub, image, title):
    plt.subplot(2, 2, sub)
    plt.imshow(image, cmap = 'gray')
    title_obj = plt.title(title)
    plt.setp(title_obj, color='w')
    plt.xticks([])
    plt.yticks([])

In [ ]:
# Cargar imagen arrows
from os import path
arrows_path = path.join("res", "arrows.jpg")
img = cv2.imread(arrows_path, 0)

- Imagen en escala de grises, formato CV_64F
- Sobel(imagen, profundidad de color, ordenes de derivada x, y, tamaño kernel)
  - Horizontal
  - Vertical
 - Laplacian(imagen, profundidad de color)

In [ ]:
# Aplicar filtros
laplacian = cv2.Laplacian(img, cv2.CV_64F)

sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)

show_image(1, img, 'Original')
show_image(2, laplacian, 'Laplacian')
show_image(3, sobel_x, 'Sobel X')
show_image(4, sobel_y, 'Sobel Y')

plt.show()

In [ ]:
# Aplicar filtros con solo 8 bits por pixel
laplacian = cv2.Laplacian(img, cv2.CV_8U)

sobel_x = cv2.Sobel(img, cv2.CV_8U, 1, 0, ksize=5)
sobel_y = cv2.Sobel(img, cv2.CV_8U, 0, 1, ksize=5)

show_image(1, img, 'Original')
show_image(2, laplacian, 'Laplacian')
show_image(3, sobel_x, 'Sobel X')
show_image(4, sobel_y, 'Sobel Y')

plt.show()

In [ ]:
import numpy as np

laplacian = cv2.Laplacian(img, cv2.CV_64F)
laplacian = np.uint8(np.absolute(laplacian))

sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
sobel_x = np.uint8(np.absolute(sobel_x))

sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
sobel_y = np.uint8(np.absolute(sobel_y))

show_image(1, img, 'Original')
show_image(2, laplacian, 'Laplacian')
show_image(3, sobel_x, 'Sobel X')
show_image(4, sobel_y, 'Sobel Y')

plt.show()

## Detección de caras
- Deep Learning
- Caffe2
  - Definición de la arquitectura del modelo https://github.com/opencv/opencv/blob/master/samples/dnn/face_detector/deploy.prototxt
  - Modelo con los pesos de las capas de RN https://github.com/opencv/opencv_3rdparty/tree/dnn_samples_face_detector_20170830
  

In [ ]:
from os import path
proto_path = path.join("res", "deploy.prototxt.txt")
model_path = path.join("res", "res10_300x300_ssd_iter_140000.caffemodel")
people_path = path.join("res", "people.jpg")

In [ ]:
# Cargar modelo en DNN de OpenCV
net = cv2.dnn.readNetFromCaffe(proto_path, model_path)

In [ ]:
# Imagen a analizar
img = cv2.imread(people_path)
(h, w) = img.shape[:2]
print((h, w))

In [ ]:
# Preprocesar imagen resize 300x300, scale factor, image_size, valores medios canales de color
blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

In [ ]:
# Enviar imagen a la red neuronal
net.setInput(blob)
detections = net.forward()

- Array de detecciones
- Umbral de confianza

In [ ]:
confidence_threshold = 0.80

for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > confidence_threshold:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        text = f"{confidence * 100:.2f}"        
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(img, (startX, startY), (endX, endY),(0, 0, 255), 2)
        cv2.putText(img, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 2)

cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyWindow('Image')